In [ ]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd
import concurrent.futures

 # Create an empty list to store the scraped data
data = []
 # Specify the URL of the page to scrape
base_url = "https://www.goodjob.life/salary-work-times/latest?p="
 # Function to scrape a single page
def scrape_page(page):
    url = base_url + str(page)
     # Send a GET request to the URL and store the response
    response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
     # Parse the HTML content of the response using BeautifulSoup
    soup = BeautifulSoup(response.content, "html.parser")
     # Find the table element containing the data
    table = soup.find("table")
     # Find the rows of the table
    rows = table.find_all("tr")
     # Loop through the rows and extract the data
    for row in rows[1:]:
        # Extract the columns of the row
        columns = row.find_all("td")
         # Extract the data from the columns
        company = columns[0].text.strip()
        job_title = columns[1].text.strip()
        work_time = columns[2].text.strip()
        overtime = columns[3].text.strip()
        salary = columns[4].text.strip()
        wage = columns[5].text.strip()
        time = columns[6].text.strip()
         # Append the data to the list
        data.append({
            "公司名稱": company,
            "職稱": job_title,
            "週工時": work_time,
            "加班頻率": overtime,
            "薪資": salary,
            "時薪": wage,
            "參考時間": time
        })
 # Number of pages to scrape
pages = 454
 # Scrape pages asynchronously using ThreadPoolExecutor
with concurrent.futures.ThreadPoolExecutor() as executor:
    # Submit scraping tasks for each page
    futures = [executor.submit(scrape_page, page) for page in range(1, pages+1)]
     # Wait for all tasks to complete
    concurrent.futures.wait(futures)
 # Create a DataFrame from the collected data
data_df = pd.DataFrame(data)
 # Save the DataFrame
data_df.to_excel("goodjob_salary.xlsx", index=False)
print("Data saved to goodjob_salary.xlsx")

# 須爬取1分20秒，有遺漏資訊